In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score
!pip install fastapi uvicorn
import fastapi
from typing import Optional

In [2]:
_x = [1]
_y = [1]
_prediction = ["True"]
_result = ["True"]
model = DummyClassifier(strategy="constant", constant="True")
model.fit(np.array([_x,_y]).reshape(1, -1), _result)

DummyClassifier(constant='True', strategy='constant')

In [3]:
def model_evaluation(y_test, y_pred):
    # Convert y_test and y_pred to boolean if they are strings
    y_test = [True if isinstance(x, str) and x == "True" else False for x in y_test]
    y_pred = [True if isinstance(x, str) and x == "True" else False for x in y_pred]

    print('Trained Model Test Data Accuracy Score :', accuracy_score(y_test, y_pred) * 100)
    testacc = accuracy_score(y_test, y_pred)
    testrecall = recall_score(y_test, y_pred, pos_label=True)  # Explicitly define pos_label as True
    testprecision = precision_score(y_test, y_pred, pos_label=True)  # Explicitly define pos_label as True
    print(' ')
    print(classification_report(y_test, y_pred))
    return testacc, testrecall, testprecision

In [4]:
def create_eval_df(x, y, result) :
    data = {"x": x, "y": y, "result": result}
    df = pd.DataFrame(data)
    eval_df = df.copy()
    eval_df["predict"] = model.predict(data[x,y])
    eval_df["correct"] = eval_df["predict"] == eval_df["result"]
    eval_df["correctandtrue"] = eval_df["predict"].isin([True]) & eval_df["result"].isin([True])
    return eval_df

In [5]:
app = fastapi.FastAPI()


@app.get("/predict/")
def predict(id:int = 0 , x:float =0, y:float=0):
    print(id, x, y)
    _x.insert(id, x)
    _y.insert(id, y)
    prediction = model.predict(np.array([x,y]).reshape(1, -1))
    _prediction.insert(id, prediction[0])
    print(prediction[0])
    return dict([("prediction", prediction[0])])

@app.get("/result/")
def result(id:int, result:str):
    print(result)
    _result.insert(id, result)
    return dict([("message", "Copied result for {}".format(id))])


In [6]:
@app.get("/retrain/")
def retrain(last_n:int):
    print ("RETRAINING over last {}".format(last_n))
    #print(_x)
    #print(_y)
    X = np.array(list(zip(_x[-last_n:], _y[-last_n:])))
    Y = np.array(_result[-last_n:])  # Keep it as a simple array

    #print(X, Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.40, stratify=Y)


    print(X_train)
    print(y_train)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    testacc, testrecall, testprecision = model_evaluation(y_test, y_pred)

    return {"message": "Model retrained", "accuracy": testacc, "recall": testrecall, "precision": testprecision}

@app.get("/new_model/")
def new_model(model:str, last_n:int, strategy:Optional[str] = None, k:Optional[int] = 0):
    print(model, last_n, strategy, k)
    if model == "Dummy" :
        model = DummyClassifier(strategy=strategy)
    elif model == "KNN" :
        model = KNeighborsClassifier(k=k)
    #model.set_params(model_params)

    message = retrain(last_n=last_n)

    returns = {"message": "Model trained {}".format(model), "train": message}
    print(returns)

    return returns
#@app.get("/evaluate_model/")
#def evaluate_model(last_n:int):
#    return {"message": "Item created", "last_n": last_n}

In [7]:
for i in range(50):
    predict(i, i*0.1, i*0.2)
    result(i, "True" if i % 2 == 0 else "False")

0 0.0 0.0
True
True
1 0.1 0.2
True
False
2 0.2 0.4
True
True
3 0.30000000000000004 0.6000000000000001
True
False
4 0.4 0.8
True
True
5 0.5 1.0
True
False
6 0.6000000000000001 1.2000000000000002
True
True
7 0.7000000000000001 1.4000000000000001
True
False
8 0.8 1.6
True
True
9 0.9 1.8
True
False
10 1.0 2.0
True
True
11 1.1 2.2
True
False
12 1.2000000000000002 2.4000000000000004
True
True
13 1.3 2.6
True
False
14 1.4000000000000001 2.8000000000000003
True
True
15 1.5 3.0
True
False
16 1.6 3.2
True
True
17 1.7000000000000002 3.4000000000000004
True
False
18 1.8 3.6
True
True
19 1.9000000000000001 3.8000000000000003
True
False
20 2.0 4.0
True
True
21 2.1 4.2
True
False
22 2.2 4.4
True
True
23 2.3000000000000003 4.6000000000000005
True
False
24 2.4000000000000004 4.800000000000001
True
True
25 2.5 5.0
True
False
26 2.6 5.2
True
True
27 2.7 5.4
True
False
28 2.8000000000000003 5.6000000000000005
True
True
29 2.9000000000000004 5.800000000000001
True
False
30 3.0 6.0
True
True
31 3.1 6.2
True

In [8]:
new_model("Dummy", 50, "stratified")

Dummy 50 stratified 0
RETRAINING over last 50
[[2.3 4.6]
 [3.5 7. ]
 [1.5 3. ]
 [4.1 8.2]
 [1.2 2.4]
 [2.9 5.8]
 [0.1 0.2]
 [4.6 9.2]
 [2.1 4.2]
 [3.9 7.8]
 [1.6 3.2]
 [1.4 2.8]
 [2.8 5.6]
 [4.  8. ]
 [3.7 7.4]
 [3.3 6.6]
 [1.1 2.2]
 [1.8 3.6]
 [4.4 8.8]
 [0.6 1.2]
 [0.5 1. ]
 [1.  2. ]
 [3.8 7.6]
 [1.9 3.8]
 [0.4 0.8]
 [4.8 9.6]
 [1.7 3.4]
 [0.7 1.4]
 [3.6 7.2]
 [3.2 6.4]]
['False' 'False' 'False' 'False' 'True' 'False' 'False' 'True' 'False'
 'False' 'True' 'True' 'True' 'True' 'False' 'False' 'False' 'True' 'True'
 'True' 'False' 'True' 'True' 'False' 'True' 'True' 'False' 'False' 'True'
 'True']
Trained Model Test Data Accuracy Score : 50.0
 
              precision    recall  f1-score   support

       False       0.00      0.00      0.00        10
        True       0.50      1.00      0.67        10

    accuracy                           0.50        20
   macro avg       0.25      0.50      0.33        20
weighted avg       0.25      0.50      0.33        20

{'message': "Model

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'message': "Model trained DummyClassifier(strategy='stratified')",
 'train': {'message': 'Model retrained',
  'accuracy': 0.5,
  'recall': 1.0,
  'precision': 0.5}}

In [9]:
@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}

In [10]:
import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())